In [1]:
import re
import requests
from bs4 import BeautifulSoup
import csv
import datetime
import sys
csv.field_size_limit(sys.maxint)
print datetime.datetime.now()
reload(sys)
sys.setdefaultencoding('utf8')
csv.field_size_limit(sys.maxint)
infile = open('test.csv','r')
reader = csv.reader(infile, delimiter ='|', quotechar = ' ')
outfile1 = open('step4_authorprofile.csv','wb')
writer1 = csv.writer(outfile1, delimiter = '|', quotechar=' ', quoting = csv.QUOTE_ALL)
writer1.writerow(['authorlink','author','fullname','othernames','authid','currentaffi','doccount','cites','coauthor','area'])
outfile2 = open('step4_error_author_links.csv','wb')
writer2 = csv.writer(outfile2, delimiter = '|', quotechar=' ', quoting = csv.QUOTE_ALL)
#wrtier2.writerow(row)
line_nb = 0
error_nb = 0
id_unique = 0
authid_list = []
for row in reader:
    line_nb += 1
    url = row[4]
    try: 
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
        #getting fullname
        titles = soup.find_all('title')
        fullname= titles[0].text.replace("Scopus - Author details (",'').replace(")","").replace("\n","")
        #getting authorID
        authorid = soup.find_all("div",{"class": "authId"})
        authid= authorid[0].text.replace("Author ID: ","").strip()
        if authid not in authid_list:
            authid_list.append(authid)
            id_unique += 1      
            #getting othernames
            names = soup.find_all("span",{"class": "displayBlock"})
            othernames = ""
            for name in names:
                othernames = othernames + name.text + ";"
    
            #getting current affiliation
            g_data = soup.find_all("div", {"class": "authAffilcityCounty"})
            for item in g_data:
                x = item.contents
                for y in x:
                    currentaffi= y.replace('\r','').replace("u'",'') 
            
            subjectarea = soup.find_all("div", {"id": "subjAreas"})
            for area in subjectarea:
                subject = area.text.replace("\n","").replace("View More","")
        
            doc = soup.find_all("div", {"id": "tabs"},{"data-count"})
            x= doc[0].text.replace('\n','').split("|")
            #getting number of documents - doccount
            y1= x[0].split(' ')
            doccount = y1[0]
            # getting number of citations - citecount
            y2= x[1].split(' ')
            citecount = y2[2]
            #getting number of coauthcount 
            y3= x[2].split(' ')
            coauthcount= y3[0]
            writer1.writerow([row[4],row[3],fullname,othernames, authid, currentaffi,doccount, citecount, coauthcount,subject])
        
    except:
        error_nb += 1
        writer2.writerow(row)
        pass
print "Done"
print "Number of error urls: ", error_nb
print "Number of unique id: ", id_unique
print datetime.datetime.now()

    

2017-02-08 15:41:35.010000
